# Remeber to update dictionary 

In [3]:
import httplib2
import json
import sys
import codecs
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np

In [4]:
ME_df = pd.read_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\F and E Series update 12020219.xlsx', sheet_name = 'MasterEligible', dtype=str)

In [3]:
ME_df

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription
0,Acura,CL,1997,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
1,Acura,CL,2000,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
2,Acura,CL,2001,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
3,Acura,CL,2002,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
4,Acura,CL,2003,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
5,Acura,CL,2004,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
6,Acura,CL,2004,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
7,Acura,CSX,2005,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
8,Acura,CSX,2006,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
9,Acura,CSX,2007,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2


In [5]:
dictionary = {460:'ford',452:'bmw',467 :'chevrolet', 448:'toyota',477:'chrysler',476:'dodge',474:'honda', 480:'infiniti', 478: 'nissan'}

In [18]:
def getMake():
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getMakeID(Make):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    MK_df = pd.DataFrame(result['Results'])
    result_df = MK_df[MK_df['Make_Name'].str.contains(Make)]
    return result_df

def getModel(Make):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMake/%s?format=json' %Make)
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getModel_Year(Make,Year):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMakeYear/make/%s/modelyear/%s?format=json' %(Make,Year))
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getModel_Year_MakeID(MakeID,Year):
    url = ('https://vpic.nhtsa.dot.gov/api//vehicles/GetModelsForMakeIdYear/makeId/%s/modelyear/%s?format=json' %(MakeID,Year))
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)

    except:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios = process.extract(str2Match,strOptions)
            Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    return Rlist
# With score
# def getExactMatch(Make_id,Year):
#     Rlist = []
#     try:
#         Valid_df = getModel_Year_MakeID(Make_id,Year)
        
#     except:
#         QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#         Rlist=['No Such year in the Database'] *len(QC_Make_year)
#     else:
#         QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#         for i in range(len(QC_Make_year.Model)):
#             str2Match = str(QC_Make_year.Model.values[i])
#             strOptions = [z for z in Valid_df.Model_Name]
#             Ratios= process.extractOne(str2Match,strOptions, scorer=fuzz.ratio)
#             Rlist.append((':'.join([ str(a) for a in list(Ratios)])))
#     return Rlist



def getQCtable_OneYear(Make_id,Year):    
    QC_Make_Year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
    QC_Make_Year['TopMatch'] = getTopMatch(Make_id,Year)
    QC_Make_Year['MatchOptions'] = getMatch(Make_id,Year)        
    QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
    QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
    QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') for i in range(len(QC_Make_Year['TopMatch']))]
    QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make_Year

def getQCtable_Allyear(Make_id):    
    QC_Make = pd.DataFrame()
    Years = list(ME_df[ME_df.Make.str.lower() == dictionary[Make_id]].Year.unique())
    for Year in Years:
        QC_Make_Year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        QC_Make_Year['TopMatch'] = getTopMatch(Make_id,Year)
        QC_Make_Year['MatchOptions'] = getMatch(Make_id,Year)        
        QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
        QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
        QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') if QC_Make_Year['HighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make

# def getHightestScores(Make_id,Year):
#     Scores = []
#     if type(getMatch(Make_id,Year)[0]) !=str:
#         for rec in getMatch(Make_id,Year):
#             Scores.append(int(rec[0].split(': ')[1]))
#     else:
#         Scores='N/A'
#     return Scores




## FORD

In [18]:
getMakeID('Ford')

,Make_ID,Make_Name
20,460,Ford
810,1298,Eagle Ford Tanks & Trailers LLC
3819,4533,Fords Trailer Sales


In [34]:
Make_id = 460

QC_Ford = getQCtable_Allyear(Make_id)

In [49]:
QC_Ford

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription,TOPexcatMatch,MatchOptions,Match,HighestScore,TopMaTchExsting
2571,Ford,500,2009,2019-09-25 16:03:15.593,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4,E-150,"[E-150: 60, F-150: 60, F-250: 60, F-350: 60, F...",No,60,Yes
2609,Ford,Crown Victoria,2009,2018-05-17 16:53:40.733,Y,round 3 list per Denise,Round 3,Crown Victoria,"[Crown Victoria: 100, Low Cab Forward: 48, Exp...",Yes,100,Yes
2640,Ford,E Series,2009,2019-09-25 16:03:15.593,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4,Escape,"[E-150: 86, E-250: 86, E-350: 86, E-450: 86, E...",No,86,Yes
2665,Ford,E-150,2009,NaT,Y,E Series update,E Series update,E-150,"[E-150: 100, F-150: 80, E-250: 80, E-350: 80, ...",Yes,100,Yes
2706,Ford,E-250,2009,NaT,Y,E Series update,E Series update,E-250,"[E-250: 100, E-150: 80, F-250: 80, E-350: 80, ...",Yes,100,Yes
2754,Ford,E-350,2009,NaT,Y,E Series update,E Series update,E-350,"[E-350: 100, E-150: 80, F-350: 80, E-250: 80, ...",Yes,100,Yes
2806,Ford,E-450,2009,NaT,Y,E Series update,E Series update,E-450,"[E-450: 100, E-150: 80, F-450: 80, E-250: 80, ...",Yes,100,Yes
2845,Ford,E-550,2009,NaT,Y,E Series update,E Series update,E-150,"[E-150: 80, F-550: 80, E-250: 80, E-350: 80, E...",No,80,Yes
2873,Ford,E-650,2009,NaT,Y,E Series update,E Series update,E-150,"[E-150: 80, F-650: 80, E-250: 80, E-350: 80, E...",No,80,Yes
2899,Ford,E-750,2009,NaT,Y,E Series update,E Series update,E-150,"[E-150: 80, F-750: 80, E-250: 80, E-350: 80, E...",No,80,Yes


In [36]:
QC_Ford_sorted = QC_Ford.sort_values(by=['Make','Model','Year'])
QC_Ford_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_single_report\\QC_Ford.xlsx',index = False)

##  BMW

In [37]:
getMakeID('BMW')

,Make_ID,Make_Name
12,452,BMW


In [38]:
Make_id = 452

QC_BMW = getQCtable_Allyear(Make_id)

In [39]:
QC_BMW_sorted = QC_BMW.sort_values(by=['Make','Model','Year'])
QC_BMW_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_single_report\\QC_BMW.xlsx',index = False)

##  Chevrolet

In [40]:
getMakeID('Chevrolet')

,Make_ID,Make_Name
27,467,Chevrolet


In [41]:
Make_id = 467

QC_Chevrolet = getQCtable_Allyear(Make_id)

In [42]:
QC_Chevrolet

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription,TopMatch,MatchOptions,Match,HighestScore,TopMaTchExsting
1222,Chevrolet,Astro,2000,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Astro Van,"[Astro Van: 90, Metro: 60, Camaro: 55, Monte C...",No,90,Yes
1228,Chevrolet,Astro Van,2000,2018-06-25 00:00:00,Y,correction to list per Denise2,Round 3,Astro Van,"[Astro Van: 100, Metro: 54, Monte Carlo: 48, S...",Yes,100,Yes
1255,Chevrolet,Blazer,2000,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Blazer,"[Blazer: 100, Tracker: 46, Silverado: 45, B7: ...",Yes,100,Yes
1272,Chevrolet,Camaro,2000,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Camaro,"[Camaro: 100, Monte Carlo: 60, Cavalier: 57, M...",Yes,100,Yes
1301,Chevrolet,Cavalier,2000,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Cavalier,"[Cavalier: 100, Camaro: 57, C6: 45, C7: 45, Ma...",Yes,100,Yes
1345,Chevrolet,Corvette,2000,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Corvette,"[Corvette: 100, Venture: 53, C6: 45, C7: 45, M...",Yes,100,Yes
1368,Chevrolet,Cruze,2000,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Corvette,"[Corvette: 54, C6: 45, C7: 45, S-10 Pickup: 40...",No,54,Yes
1423,Chevrolet,Express 1500,2000,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Express,"[Express: 90, S-10: 68, W5: 45, T-Series: 45, ...",No,90,No
1437,Chevrolet,Express 2500,2000,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Express,"[Express: 90, W5: 45, T-Series: 45, S-10: 45, ...",No,90,No
1476,Chevrolet,Impala,2000,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Impala,"[Impala: 100, Malibu: 50, Silverado: 45, Monte...",Yes,100,Yes


In [43]:
QC_Chevrolet_sorted = QC_Chevrolet.sort_values(by=['Make','Model','Year'])
QC_Chevrolet_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_single_report\\QC_Chevrolet.xlsx',index = False)

##  Toyota

In [44]:
getMakeID('Toyota')

,Make_ID,Make_Name
8,448,Toyota


In [45]:
Make_id = 448

QC_Toyota = getQCtable_Allyear(Make_id)

In [46]:
QC_Toyota_sorted = QC_Toyota.sort_values(by=['Make','Model','Year'])
QC_Toyota_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_single_report\\QC_Toyota.xlsx',index = False)

##  Chrysler

In [47]:
getMakeID('Chrysler')

,Make_ID,Make_Name
37,477,Chrysler
891,1383,DaimlerChrysler Commercial Bus


In [48]:
Make_id = 477

QC_Chrysler = getQCtable_Allyear(Make_id)

In [49]:
QC_Chrysler_sorted = QC_Chrysler.sort_values(by=['Make','Model','Year'])
QC_Chrysler_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_single_report\\QC_Chrysler.xlsx',index = False)

##  Dodge

In [50]:
getMakeID('Dodge')

,Make_ID,Make_Name
36,476,Dodge
132,580,Sprinter (Dodge or Freightliner)
928,1422,Dodgen Industries
7067,8047,Dodger Trailer Manufacturing Inc


In [51]:
Make_id = 476

QC_Dodge = getQCtable_Allyear(Make_id)

In [52]:
QC_Dodge_sorted = QC_Dodge.sort_values(by=['Make','Model','Year'])
QC_Dodge_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_single_report\\QC_Dodge.xlsx',index = False)

##  Honda

In [53]:
getMakeID('Honda')

,Make_ID,Make_Name
34,474,Honda
6408,7351,Sundiro Honda Motorcycle Co. Ltd


In [54]:
Make_id = 474

QC_Honda = getQCtable_Allyear(Make_id)

In [55]:
QC_Honda_sorted = QC_Honda.sort_values(by=['Make','Model','Year'])
QC_Honda_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_single_report\\QC_Honda.xlsx',index = False)

## Infiniti

In [5]:
getMakeID('Infiniti')

,Make_ID,Make_Name
3984,480,Infiniti


In [8]:
Make_id = 480

QC_Infiniti = getQCtable_Allyear(Make_id)

In [9]:
QC_Infiniti

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription,TopMatch,MatchOptions,Match,HighestScore,TopMaTchExsting
4901,Infiniti,45,2004,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Q45,"[Q45: 90, M45: 90, FX45: 90, G35: 45, FX35: 45]",No,90,Yes
4923,Infiniti,FX,2004,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,FX35,"[FX35: 90, FX45: 90, QX56: 45, Q45: 0, G35: 0]",No,90,Yes
4938,Infiniti,FX35,2004,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,FX35,"[FX35: 100, FX45: 75, G35: 57, I35: 57, QX56: 50]",Yes,100,Yes
4953,Infiniti,FX45,2004,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,FX45,"[FX45: 100, FX35: 75, Q45: 57, M45: 57, QX56: 50]",Yes,100,Yes
4967,Infiniti,FX50,2004,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,FX35,"[FX35: 75, FX45: 75, QX56: 50, Q45: 29, G35: 29]",No,75,Yes
4982,Infiniti,G,2004,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,G35,"[G35: 90, Q45: 0, M45: 0, FX35: 0, FX45: 0]",No,90,Yes
5009,Infiniti,G20,2004,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,G35,"[G35: 33, Q45: 0, M45: 0, FX35: 0, FX45: 0]",No,33,Yes
5015,Infiniti,G35,2004,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,G35,"[G35: 100, I35: 67, FX35: 57, Q45: 33, M45: 33]",Yes,100,Yes
5044,Infiniti,I30,2004,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,I35,"[I35: 67, G35: 33, FX35: 29, Q45: 0, M45: 0]",No,67,Yes
5052,Infiniti,I35,2004,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,I35,"[I35: 100, G35: 67, FX35: 57, Q45: 33, M45: 33]",Yes,100,Yes


In [16]:
QC_Infiniti_sorted = QC_Infiniti.sort_values(by=['Make','Model','Year'])
QC_Infiniti_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_single_report\\QC_Infiniti.xlsx',index = False)

## Nissan

In [7]:
getMakeID('Nissan')

,Make_ID,Make_Name
5717,478,Nissan


In [19]:
Make_id = 478

QC_Nissan = getQCtable_Allyear(Make_id)

In [15]:
QC_Nissan

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription,TopMatch,MatchOptions,Match,HighestScore,TopMaTchExsting
8416,NISSAN,Infiniti,2010,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,Frontier,"[Frontier: 50, Titan: 36, Pathfinder: 33, Alti...",No,50,No
8201,Nissan,350,2010,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3,370Z,"[370Z: 57, GT-R: 0, VERSA: 0, Versa: 0, Altima...",No,57,No
8224,Nissan,350Z,2010,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,370Z,"[370Z: 75, GT-R: 0, VERSA: 0, Versa: 0, Altima...",No,75,No
8243,Nissan,370Z,2010,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,370Z,"[370Z: 100, GT-R: 0, VERSA: 0, Versa: 0, Altim...",Yes,100,Yes
8244,Nissan,370Z,2010,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,370Z,"[370Z: 100, GT-R: 0, VERSA: 0, Versa: 0, Altim...",Yes,100,Yes
8274,Nissan,Altima,2010,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,Altima,"[Altima: 100, Maxima: 67, Titan: 55, Armada: 5...",Yes,100,Yes
8288,Nissan,Altima Coupe,2010,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,Altima,"[Altima: 90, Maxima: 60, Rogue: 54, Titan: 54,...",No,90,Yes
8312,Nissan,Armada,2010,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,Armada,"[Armada: 100, Altima: 50, Maxima: 50, VERSA: 3...",Yes,100,Yes
8331,Nissan,Cube,2010,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,Cube,"[Cube: 100, Rogue: 44, VERSA: 22, Versa: 22, P...",Yes,100,Yes
8346,Nissan,Equator,2010,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2,Sentra,"[Sentra: 46, Murano: 46, GT-R: 45, Pathfinder:...",No,46,No


In [20]:
QC_Nissan_sorted = QC_Nissan.sort_values(by=['Make','Model','Year'])
QC_Nissan_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_single_report\\QC_Nissan_.xlsx',index = False)